In [8]:
import numpy as np
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score,roc_auc_score, classification_report
import warnings
warnings.filterwarnings('ignore')
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
import torch
import torch.nn.functional as F
from transformers.utils import logging
from transformers import TrainerCallback
from transformers.utils import logging
import time
import os

In [3]:
os.environ["WANDB_DISABLED"] = "true"

In [4]:
# 1. Load your SnapFood dataset
df = pd.read_csv("https://raw.githubusercontent.com/samyvivo/Snap_Food_Sentiment_Analysis_fa/refs/heads/main/snappfood_sentiment_train.csv")

In [6]:
df = df[["comment", "label"]].dropna()
df["label"] = np.where(df["label"] == "SAD", 0, 1)
df["label"] = df["label"].astype(int)

In [9]:
persian_punc = "؟،؛«»ـ"
all_punc = string.punctuation + persian_punc

def clean(text):
    text = str(text)
    text = str(text).lower()
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)  # remove non-Persian characters
    text = re.sub("\[.*?\]", "", text)
    text = re.sub("<.*?>+", "", text)
    text = re.sub(f"[{re.escape(all_punc)}]", "", text)
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub("\n", "", text)
    text = re.sub("\w*\d\w*", "", text)
    text = text.replace("—", "")
    text = " ".join(text)
    return text

In [10]:
df["comment"] = df["comment"].apply(clean)

In [11]:
X = df["comment"].tolist()
y = df["label"].tolist()

# 2. Split dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [12]:
# 3. Tokenize using BERT tokenizer
model_name = "HooshvareLab/bert-fa-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# 4. Convert to HuggingFace Dataset format
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "label": train_labels
})
test_dataset = Dataset.from_dict({
    "input_ids": test_encodings["input_ids"],
    "attention_mask": test_encodings["attention_mask"],
    "label": test_labels
})

In [ ]:
# 5. Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

In [ ]:
# 6. Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
# 7. Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Get predicted class labels
    preds = torch.argmax(torch.tensor(logits), axis=1)

    # Apply softmax to get probabilities
    probs = F.softmax(torch.tensor(logits), dim=1)[:, 1]  # Probability of class 1

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "roc_auc": roc_auc_score(labels, probs)
    }

In [ ]:
# 8. Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# 9. Train the model
trainer.train()

In [ ]:
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")

In [ ]:
trainer.save_model(r"C:/Users/samyh/Desktop/BERT/snapfood-bert-model")

# Save tokenizer
tokenizer.save_pretrained(r"C:/Users/samyh/Desktop/BERT/snapfood-bert-model")

In [ ]:
import shutil

# Save
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")

# Zip the directory
shutil.make_archive("snapfood_roberta_model", 'zip', "./saved_model")